# 1. 

Реализуйте алгоритм Symspell - https://medium.com/@wolfgarbe/1000x-faster-spelling-correction-algorithm-2012-8701fcd87a5f. Он похож на алгоритм Норвига, но проще и быстрее. Там к словам в словаре применяется только одна операция - удаление символа. Чтобы найти исправление из слова тоже удаляются символы и сравниваются с теми, что хранятся в словаре. Оцените качество полученного алгоритма теми же тремя метриками.

In [1]:
import os, re
from string import punctuation
import numpy as np
import json
from collections import Counter, defaultdict
from pprint import pprint
from nltk import sent_tokenize
punctuation += "«»—…“”"
punct = set(punctuation)
from sklearn.metrics import classification_report, accuracy_score
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from collections import Counter

def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text


def preprocess(text):
    sents = sentenize(text)
    return [normalize(sent.text) for sent in sents]

def ngrammer(tokens, n):
    ngrams = []
    tokens = [token for token in tokens]
    for i in range(0,len(tokens)-n+1):
        ngrams.append(tuple(tokens[i:i+n]))
    return ngrams

In [2]:
bad = open('data/sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('data/correct_sents.txt', encoding='utf8').read().splitlines()

In [3]:
# напишем функцию, которая будет сопоставлять слова в правильном и ошибочном варианте
# разобьем предложение по пробелам и удалим пунктуация на границах слов
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_1 if (set(token)-punct)]
    tokens_2 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_2 if (set(token)-punct)]
    
    return list(zip(tokens_1, tokens_2))

In [4]:
pprint(align_words(true[1], bad[1]))

[('апофеозом', 'опофеозом'),
 ('дня', 'дня'),
 ('для', 'для'),
 ('меня', 'меня'),
 ('сегодня', 'сегодня'),
 ('стала', 'стала'),
 ('фраза', 'фраза'),
 ('услышанная', 'услышанная'),
 ('в', 'в'),
 ('новостях', 'новостях')]


In [5]:
corpus = open('data/wiki_data.txt', encoding='utf8').read()

In [6]:
# создаем словарь
vocab = set(re.findall('\w+', corpus.lower()))


In [7]:
def predict_mistaken(word, vocab):

    if word in vocab:
        return 0
    else:
        return 1

    
    

In [8]:
WORDS = Counter(re.findall('\w+', corpus.lower()))

In [9]:
# фунцкия расчета вероятности слова
N = sum(WORDS.values())
def P(word, N=N): 
    "Вычисляем вероятность слова"
    return WORDS[word] / N


In [10]:
def correction(word, precalculation_dict): 
    "Находим наиболее вероятное похожее слово"
    return max(candidates(word, precalculation_dict), key=P)

def candidates(word, precalculation_dict): 
    "Генерируем кандидатов на исправление"
    '''Generate terms with an edit distance (deletes only) from the input term and search them in the dictionary. 
    For a word of length n, an alphabet size of a, an edit distance of 1, there will be just n deletions, 
    for a total of n terms at search time.'''
    candidates = list()
    candidates.append(word)
    candidates.extend(precalculation_dict[word])
    deletes = edits1(word)
    for one_delete in deletes:
        candidates.extend(precalculation_dict[one_delete])
    return candidates

def edits1(word):
    "Создаем кандидатов, которые отличаются на одну букву"
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    return set(deletes)

"Generate terms with an edit distance (deletes only) from each dictionary term and add them together with the original term to the dictionary. This has to be done only once during a pre-calculation step."

In [11]:
precalculation_dict = defaultdict(list)

for word in vocab:
    deletes = edits1(word)
    for one_delete in deletes:
        precalculation_dict[one_delete].append(word)

In [12]:
precalculation_dict['солнц']

['солнца', 'солнце', 'солнцу']

In [13]:
%%time
correction('сонце', precalculation_dict)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 87.7 µs


'конце'

In [14]:
%%time
correction('опофеоз', precalculation_dict)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 52.9 µs


'апофеоз'

Для оценки используем будем использовать три метрики:  
1) процент правильных слов;  
2) процент исправленных ошибок  
3) процент ошибочно исправленных правильных слов

In [18]:
correct = 0
total = 0

total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

cashed = {}
for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        # чтобы два раза не исправлять одно и тоже слово - закешируем его
        # перед тем как считать исправление проверим нет ли его в кеше
        predicted = cashed.get(pair[1], correction(pair[1], precalculation_dict))
        cashed[pair[1]] = predicted
        
        
        if predicted == pair[0]:
            correct += 1
        total += 1
        
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] !=  predicted:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == predicted:
                mistaken_fixed += 1
        
    if not i % 100:
        print(i)
        

0
100
200
300
400
500
600
700
800
900


In [19]:
print(correct/total) 
print(mistaken_fixed/total_mistaken) 
print(correct_broken/total_correct) 

0.4312687312687313
0.3200306983883346
0.5520845296887562


На семинаре: (пересчитала с исправленной строчкой cashed[pair[1]] = predicted)

0.8693306693306694

0.5042210283960092

0.07603077983231882

Таким образом, удалось исправить меньше ошибок (32 < 50), меньше слов было подобрано правильно (43 < 87) и было сделано больше ошибочных исправлений (55 > 8). Но текущий алгоритм работает значительно быстрее. 

# 2. 
Добавьте к полученному алгоритму исправления (Symspell) триграммную модель и проверьте, улучшает ли она качество. Триграммную модель нужно вставить туда, где у вас выбирается один из нескольких кандидатов на исправление.

In [20]:
corpus_wiki = [['<start>', '<start>'] + sent + ['<end>'] for sent in preprocess(corpus)]

In [21]:
def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [22]:
unigrams = Counter()
bigrams = Counter()
trigrams = Counter()

for sentence in corpus_wiki:
    unigrams.update(sentence)
    bigrams.update(ngrammer(sentence))
    trigrams.update(ngrammer(sentence, 3))


In [27]:
# оцените качество также как и раньше
mistakes = []
total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

total = 0
correct = 0



for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    
    word_pairs = [('<start>', '<start>')] + word_pairs
    pred_sent = []
    for j in range(2, len(word_pairs)):
        
        pred = None
        
        # проверяем, что слова нет в словаре, чтобы не исправлять все слова
        if not predict_mistaken(word_pairs[j][1], WORDS):
            pred = word_pairs[j][1]
            
        
        else:
            # находим кандидатов для исправления
            predicted = candidates(word_pairs[j][1], precalculation_dict) 
        
            # берем два предыдущих слова для контекста
            prev_word = word_pairs[j-2][1] + ' ' + word_pairs[j-1][1] 
        
            # проверяем есть ли биграмма из двух предыдуших слов, если да - вычисляем триграмму
            # если нет, проверяем, есть ли одно из предыдуших слов в униграммах, если да - вычисляем биграмму
            # если нет, остается только взять первое по близости
            if prev_word not in bigrams:
                
                prev_word = word_pairs[j-1][1]
                if prev_word not in unigrams:
                    pred = predicted[0][0]
            
        
                else:
                    #
                    lm_predicted = []
                    for word in predicted:
                        bigram = ' '.join([prev_word, word])
                        # домножаем полученную метрику для слова на вероятность биграма
                        # биграм - предыдущее слово + текущее слово кандидат
                        # 1 тут для того чтобы не получались 
                        lm_predicted.append((word, ((bigrams[bigram]/unigrams[prev_word])))) 


                    if lm_predicted:

                        pred = sorted(lm_predicted, key=lambda x: -x[1])[0][0]
            else:
                #
                lm_predicted = []
                for word in predicted:
                    trigram = ' '.join([prev_word, word])
                    lm_predicted.append((word, (trigrams[trigram]/bigrams[prev_word]))) 
    
    
                if lm_predicted:
            
                  pred = sorted(lm_predicted, key=lambda x: -x[1])[0][0]

        
        if pred is None:
            pred = word_pairs[j][1]
        

        
        if pred == word_pairs[j][0]:
            correct += 1
        else:
            mistakes.append((word_pairs[j][0], word_pairs[j][1], pred))
        total += 1
            
        if word_pairs[j][0] == word_pairs[j][1]:
            total_correct += 1
            if word_pairs[j][0] !=  pred:
                correct_broken += 1
        else:
            total_mistaken += 1
            if word_pairs[j][0] == pred:
                mistaken_fixed += 1
    
    if not i % 100:
        print(i)

0
100
200
300
400
500
600
700
800
900


In [28]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.9026830877501649
0.04832214765100671
0.021080368906455864


Значительно увеличился процент правильных слов (43 < 90), но также значительно меньше слов было подобрано правильно (32 > 5), но и процент неверных исправлений стал меньше (55 > 2)